In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import notebook
from pathlib import Path

In [2]:
averaged_spectra = pd.read_csv("outputFiles/averaged_spectra_transformed.csv", engine="c")

In [3]:
averaged_spectra =averaged_spectra.rename(columns={"Unnamed: 0":"sample_code"})

In [4]:
PATH_TO_PREV_SPLITS = "./v2.2/splits"
PATH_TO_CURR_SPLITS = "./outputFiles/splits"

In [5]:
for path in Path(PATH_TO_CURR_SPLITS).rglob("*.csv"):
    _ = pd.read_csv(path)
    try:
        _ = _.rename(columns={"index":"x"})
        _.to_csv(path)
    except Exception as e:
        print(e)
for path in Path(PATH_TO_PREV_SPLITS).rglob("*.csv"):
    _ = pd.read_csv(path)
    try:
        _ = _.rename(columns={"index":"x"})
        _.to_csv(path)
    except Exception as e:
        print(e)

In [6]:
df = pd.DataFrame()

In [7]:
chemicals =['boron', 'calcium', 'clay', 'copper', 'ec_salts',
       'exchangeable_acidity', 'iron', 'magnesium', 'manganese', 'phosphorus',
       'potassium', 'sand', 'silt', 'sodium', 'sulphur', 'zinc', 'ph']


In [8]:
try:
    os.mkdir("previous_model_splits")
except Exception as e:
    print(e)

[WinError 183] Cannot create a file when that file already exists: 'previous_model_splits'


In [29]:
for path in Path(PATH_TO_PREV_SPLITS).rglob("*.csv"):
    p = pd.read_csv(path)
    d = averaged_spectra[['sample_code']].rename(columns={"sample_code":"x"})
    p = p.loc[(p['x'].isin(averaged_spectra['sample_code']))]
    p.to_csv(f"previous_model_splits/{path.name}")

In [10]:
import rpy2.robjects as robjects
r = robjects.r
r['source']('splits.R')
split = robjects.globalenv['split']
from rpy2.robjects import pandas2ri

try:
    os.mkdir("./outputFiles/splits")
    os.mkdir("./outputFiles/rds")
except:
    print()


pandas2ri.activate()

split("D://CropNutsDocuments/DS-ML94/outputFiles/transformedWavelet.csv","D://CropNutsDocuments/DS-ML94/outputFiles/rds/", "D://CropNutsDocuments/DS-ML94/outputFiles/splits")

R[write to console]: prospectr version 0.2.6 -- 'chicago'

R[write to console]: check the github repository at: https://github.com/l-ramirez-lopez/prospectr/

R[write to console]: data.table 1.14.6 using 4 threads (see ?getDTthreads).  Latest news: r-datatable.com



[1] "chemical : exchangeable_acidity"


R[write to console]: |--------------------------------------------------|
|
R[write to console]: =====
R[write to console]: ===
R[write to console]: ==
R[write to console]: ===
R[write to console]: ==
R[write to console]: ==
R[write to console]: ===
R[write to console]: ==
R[write to console]: ===
R[write to console]: ==
R[write to console]: ==
R[write to console]: ===
R[write to console]: ==
R[write to console]: ==
R[write to console]: ===
R[write to console]: ==
R[write to console]: ===
R[write to console]: ==
R[write to console]: ==
R[write to console]: ==
R[write to console]: |

R[write to console]: Error: cons memory exhausted (limit reached?)

R[write to console]: Error: no more error handlers available (recursive errors?); invoking 'abort' restart



RRuntimeError: Error: cons memory exhausted (limit reached?)


In [34]:
for path in Path(PATH_TO_CURR_SPLITS).rglob("*.csv"):
    filename = str(path.name)
    chemical = filename.split("_")[0]
    train_test_valid = filename.split("_")[1]
#     curr = pd.read_csv(path)
#     print(len(curr.loc[~(curr['x'].isin(averaged_spectra['sample_code']))]))
#     prev = pd.read_csv(f"previous_model_splits/{filename}")
#     new = pd.concat([curr, prev])[['x']].drop_duplicates()
#     print(len(new.loc[~(new['x'].isin(averaged_spectra['sample_code']))]))
#     new = new.loc[(new['x'].isin(averaged_spectra['sample_code']))]
#     new.to_csv(f"{path}")
    for path in Path(PATH_TO_CURR_SPLITS).rglob(f"{chemical}*.csv"):
        if(train_test_valid not in str(path.name)):
            df = pd.read_csv(path)
            df = df.loc[~(df['x'].isin(new['x']))]

    

outputFiles\splits\boron_test_sample_codes.csv
outputFiles\splits\boron_train_sample_codes.csv
outputFiles\splits\boron_valid_sample_codes.csv


In [ ]:
import psycopg2
import pandas as pd
import pyodbc

In [ ]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                        "Server=192.168.5.18\CROPNUT;"
                        "Database=cropnuts;"
                        "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [ ]:
wetchem_df = pd.read_csv("outputFiles/lv_filtered.csv")

In [ ]:
lims_df = pd.read_sql(f"SELECT client_name, sample_code from SampleResults WHERE sample_code IN {tuple(wetchem_df.sample_code.values)}", con=conn_lims)

In [ ]:
lims_df.dropna()

In [ ]:
for client in lims_df.client_name.unique():
    client = str(client).strip()
    try:
        os.mkdir(f"outputFiles/clients/{client}")
    except Exception as e:
        (e)
    try:
        client_df = pd.DataFrame()
        client_df['sample_code'] = lims_df.loc[lims_df['client_name'] == str(client).strip()]['sample_code'].str.strip().unique()
        if(len(client_df.index) < 10):
            os.rmdir(f"outputFiles/clients/{client}")
        else:
            client_df.to_csv(f"outputFiles/clients/{client}/{client}.csv")
    except Exception as e:
        print(e)

In [ ]:
averaged_spectra = averaged_spectra.rename(columns={"Unnamed: 0":"sample_code"})

In [ ]:
for client in os.listdir("outputFiles/clients"):
    if(client == "Apollo_Kenya"):
        continue
    df = pd.read_csv(f"outputFiles/clients/{client}/{client}.csv")
    df = pd.merge(df, averaged_spectra, on="sample_code", how="left")
    try:
        df = df.drop("Unnamed: 0", axis=1)
    except:
        print()
    df = df.set_index("sample_code")
    df.to_csv(f"outputFiles/clients/{client}/{client}.csv")    

In [ ]:
def get_db_cursor():
    username = "doadmin"
    password = 'yzmodwh2oh16iks6'
    host = 'db-postgresql-cl1-do-user-2276924-0.db.ondigitalocean.com'
    port = 25060
    database = 'MandatoryMetadata'
    schema = 'historical'

    conn = psycopg2.connect(host=host,database=database, user=username, password=password, port=port)
    cur = conn.cursor()
    cur.execute("SET search_path TO " + schema)

    return conn, cur

In [ ]:
conn, cur = get_db_cursor()

In [ ]:
apolo = pd.read_csv("outputFiles/clients/Apollo_Kenya/Apollo_Kenya.csv")

In [ ]:
metadata = pd.read_sql(f"SELECT metadata_id, sample_code FROM mandatorymetadata WHERE sample_code IN {tuple(apolo.sample_code.unique())}", con=conn)

In [ ]:
metadata.sample_code.unique()

In [ ]:
spectra = pd.read_sql(f"SELECT metadata_id, averaged_spectra, non_averaged_spectra FROM spectraldata WHERE metadata_id IN {tuple(metadata.metadata_id.unique())}", con=conn)


In [ ]:
spectra = pd.merge(metadata, spectra, on="metadata_id", how="inner")

In [ ]:
spectra = spectra.dropna(subset =['averaged_spectra'])[['sample_code','averaged_spectra']]

In [ ]:
spectra = spectra.set_index("sample_code")

In [ ]:
spectra

In [ ]:
_ = spectra.to_dict()

In [ ]:
_['averaged_spectra'].keys()

In [ ]:
df_2 = pd.DataFrame()

In [ ]:
df_2.index = _['averaged_spectra'].keys()

In [ ]:
df_2

In [ ]:
import ast
import numpy as np
for sample in list(_['averaged_spectra'].keys()):
    print(sample)
    spectra = _['averaged_spectra'][sample]
#     print(spectra['521'])
#     spectra = ast.literal_eval(spectra)
    key = list(spectra.keys())[0]
    spectra_list =  spectra[key]
    print(len(spectra_list))
    for spectra_val,column in zip(spectra_list, np.arange(0, len(spectra_list)*2, 2)):
        df_2.loc[df_2.index == sample, column+522] = spectra_val

In [ ]:
df_2 = df_2.reset_index()

In [ ]:
df_2=df_2.rename(columns={"index":"sample_code"})

In [ ]:
df_2.to_csv("outputFiles/clients/Apollo_Kenya/Apollo_Kenya.csv")

In [ ]:
import rpy2.robjects as robjects
r = robjects.r
r['source']('splits.R')
split = robjects.globalenv['split']
from rpy2.robjects import pandas2ri


pandas2ri.activate()

In [ ]:
for client in os.listdir("outputFiles/clients"):
    print(client)
    try:
        os.mkdir(f"outputFiles/clients/{client}/rds")
        os.mkdir(f"outputFiles/clients/{client}/splits")
    except Exception as e:
        print(e) 
    print()
    split(f"D://CropNutsDocuments/DS-ML94/outputFiles/clients/{client}/{client}.csv",f"D://CropNutsDocuments/DS-ML94/outputFiles/clients/{client}/rds/", f"D://CropNutsDocuments/DS-ML94/outputFiles/clients/{client}/splits" )